In [25]:
from nibabel.freesurfer.io import read_geometry
import numpy as np
from pathlib import Path
import os
import pandas as pd
import re

In [12]:
def compute_surface_area(vertices, faces):
    """
    Compute total surface area of a triangulated mesh (vectorized).
    
    Parameters:
    -----------
    vertices : array (N, 3)
        Vertex coordinates
    faces : array (M, 3)
        Triangle faces (indices into vertices)
    
    Returns:
    --------
    total_area : float
        Total surface area in mm² (if vertices are in mm)
    """
    # Get all vertices for each face at once
    v0 = vertices[faces[:, 0]]  # (M, 3)
    v1 = vertices[faces[:, 1]]  # (M, 3)
    v2 = vertices[faces[:, 2]]  # (M, 3)
    
    # Compute edge vectors for all triangles
    edge1 = v1 - v0  # (M, 3)
    edge2 = v2 - v0  # (M, 3)
    
    # Cross product for all triangles
    cross_products = np.cross(edge1, edge2)  # (M, 3)
    
    # Area of each triangle
    triangle_areas = 0.5 * np.linalg.norm(cross_products, axis=1)  # (M,)
    
    # Total area
    total_area = np.sum(triangle_areas)
    
    return total_area


def sample_surface(vertices, faces, n_samples):
    """Sample points uniformly on a triangulated surface."""
    # Compute face areas (vectorized)
    v0 = vertices[faces[:, 0]]
    v1 = vertices[faces[:, 1]]
    v2 = vertices[faces[:, 2]]
    
    cross_products = np.cross(v1 - v0, v2 - v0)
    areas = 0.5 * np.linalg.norm(cross_products, axis=1)
    
    # Sample faces proportional to area
    face_probs = areas / areas.sum()
    sampled_faces = np.random.choice(len(faces), n_samples, p=face_probs)
    
    # Sample points within each face (barycentric coordinates)
    r1, r2 = np.random.random(n_samples), np.random.random(n_samples)
    sqrt_r1 = np.sqrt(r1)
    
    points = []
    normals = []
    for i, face_idx in enumerate(sampled_faces):
        face = faces[face_idx]
        v0, v1, v2 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        
        # Random point in triangle
        point = (1 - sqrt_r1[i]) * v0 + sqrt_r1[i] * (1 - r2[i]) * v1 + sqrt_r1[i] * r2[i] * v2
        points.append(point)
        
        # Face normal
        normal = np.cross(v1 - v0, v2 - v0)
        normal /= np.linalg.norm(normal)
        normals.append(normal)
    
    return np.array(points), np.array(normals)

def compute_surface_dose(source_vertices, source_faces, 
                         target_vertices, target_faces,
                         n_samples_source=1000, n_samples_target=1000):
    """
    Monte Carlo estimation of surface-to-surface dose.
    """
    # Sample points on both surfaces
    source_points, source_normals = sample_surface(
        source_vertices, source_faces, n_samples_source
    )
    
    target_points, target_normals = sample_surface(
        target_vertices, target_faces, n_samples_target
    )
    
    # Compute areas (needed for normalization)
    source_area = compute_surface_area(source_vertices, source_faces)
    target_area = compute_surface_area(target_vertices, target_faces)
    
    # Compute all pairwise distances
    from scipy.spatial.distance import cdist
    distances = cdist(target_points, source_points)
    
    # Integrate
    total_dose = 0
    for i, target_pt in enumerate(target_points):
        for j, source_pt in enumerate(source_points):
            r_vec = target_pt - source_pt
            d = distances[i, j]
            
            if d < 1e-6:  # Skip if too close
                continue
                
            r_hat = r_vec / d
            
            # Cosine factors
            cos_source = np.dot(source_normals[j], r_hat)
            cos_target = np.dot(target_normals[i], -r_hat)
            
            # Only count if both angles are favorable
            if cos_source > 0 and cos_target > 0:
                contribution = (cos_source * cos_target) / (d**2)
                total_dose += contribution
    
    # Normalize by number of samples (Monte Carlo integration)
    total_dose *= (source_area * target_area) / (n_samples_source * n_samples_target)
    
    return total_dose

In [37]:
thalamic_nuclei = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]

hips_thomas_ref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="index"
)["struct"]
hips_thomas_invref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="struct"
)["index"]

new_index = {ind: re.sub(r"(\w+)_(\d+)", r"\2-\1", ind) for ind in hips_thomas_ref}
new_index['MD_Pf_12'] = "12-MD-Pf"
new_ref = hips_thomas_invref.rename(index=new_index)

In [43]:
new_ref.index

Index(['1-THALAMUS', '2-AV', '4-VA', '5-VLa', '6-VLP', '7-VPL', '8-Pul',
       '9-LGN', '10-MGN', '11-CM', '12-MD-Pf', '13-Hb', '14-MTT', '26-Acc',
       '27-Cau', '28-Cla', '29-GPe', '30-GPi', '31-Put', '32-RN', '33-GP',
       '34-Amy'],
      dtype='object', name='struct')

In [48]:
new_ref['2-AV']

np.int64(2)

In [49]:
new_ref2 = pd.Series({new_ref[struct]: struct for struct in new_ref.index})
new_ref2

1     1-THALAMUS
2           2-AV
4           4-VA
5          5-VLa
6          6-VLP
7          7-VPL
8          8-Pul
9          9-LGN
10        10-MGN
11         11-CM
12      12-MD-Pf
13         13-Hb
14        14-MTT
26        26-Acc
27        27-Cau
28        28-Cla
29        29-GPe
30        30-GPi
31        31-Put
32         32-RN
33         33-GP
34        34-Amy
dtype: object

In [51]:
surf_dir = Path("/mnt/c/Users/srs-9/Dev/ms_mri/data/fastsurfer/MNI152_T1_1mm1/MNI152_T1_1mm1/surf")

choroid_surf = surf_dir / "lh.choroid"
cp_vertices, cp_faces = read_geometry(choroid_surf)

pul_surf = surf_dir / "lh.8-Pul"
pul_vertices, pul_faces = read_geometry(pul_surf)

av_surf = surf_dir / "lh.2-AV"
av_vertices, av_faces = read_geometry(av_surf)

md_surf = surf_dir / "lh.12-MD-Pf"
md_vertices, md_faces = read_geometry(md_surf)

In [50]:
vertices = {}
faces = {}
for struct in new_ref2[thalamic_nuclei]:
    surf = surf_dir / f"lh.{struct}"
    vertices[struct], faces[struct] = read_geometry(surf)

In [53]:
compute_surface_area(vertices['4-VA'], faces['4-VA'])

np.float64(281.91186363334475)

In [54]:
surf_doses = {}
for key in vertices.keys():
    surf_doses[key] = compute_surface_dose(cp_vertices, cp_faces, vertices[key], faces[key])

In [55]:
surf_doses

{'2-AV': np.float64(24.342988845571654),
 '4-VA': np.float64(26.193953474645046),
 '5-VLa': np.float64(8.8843097367949),
 '6-VLP': np.float64(95.00188394862035),
 '7-VPL': np.float64(58.22615374631781),
 '8-Pul': np.float64(197.6747853695808),
 '9-LGN': np.float64(12.385671799869824),
 '10-MGN': np.float64(7.785687340754838),
 '11-CM': np.float64(17.442323420917603),
 '12-MD-Pf': np.float64(50.77035073401767)}

In [11]:
print(compute_surface_area(cp_vertices, cp_faces))
print(compute_surface_area(pul_vertices, pul_faces))
print(compute_surface_area(av_vertices, av_faces))


654.1968707803137
805.450473748069
110.01189187523484


In [15]:
compute_surface_dose(cp_vertices, cp_faces, pul_vertices, pul_faces)

np.float64(212.21204649273653)

In [14]:
compute_surface_dose(cp_vertices, cp_faces, av_vertices, av_faces)

np.float64(19.53722465285752)

In [18]:
compute_surface_dose(cp_vertices, cp_faces, md_vertices, md_faces)

np.float64(50.71177789086865)